In [1]:
# identifier: yt-cleaned-data
# Master username: admin
# Master password: Admin123
# new vpc security group name: yt_demo_vpc
# Database port: 3306
# initial database name: yt_cleaned_data

In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary datasets
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Get English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Lemmatize and remove stopwords
    processed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]

    # Join the processed words back into a string
    return ' '.join(processed_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
pip install pandas sqlalchemy mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
host = ''  # Replace with your actual AWS RDS endpoint
database = 'yt_cleaned_data'
user = 'admin'
password = 'Admin123'
port = 3306

# Create SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# Read data from MySQL table into a DataFrame
df = pd.read_sql('SELECT * FROM yt_cleaned_data', con=engine)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Your data
data = list(df['tags_list'])

documents = [preprocess_text(text) for text in data]

# Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents).toarray()

# Perform hierarchical clustering
Z = linkage(X, method='ward')

# Choose the number of clusters
k = 250
labels = fcluster(Z, k, criterion='maxclust')

In [7]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(X, labels)
print(f"Silhouette Score: {silhouette_avg}")


Silhouette Score: 0.3783247052214623


In [8]:
import pickle

with open('clustering_labels.pkl', 'wb') as f:
    pickle.dump(labels, f)

# # Save the TF-IDF vectorized matrix
# with open('tfidf_matrix.pkl', 'wb') as f:
#     pickle.dump(X, f)

# Save the vectorizer itself
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [9]:
# Load the clustering labels
with open('clustering_labels.pkl', 'rb') as f:
    labels = pickle.load(f)

# # Load the TF-IDF vectorized matrix
# with open('tfidf_matrix.pkl', 'rb') as f:
#     X = pickle.load(f)

# Load the vectorizer
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)


In [10]:
for i in range(12):
    z = X[i*400:i*400+400]
    # Save the TF-IDF vectorized matrix
    with open(f'tfidf_matrix_{i}.pkl', 'wb') as f:
        pickle.dump(z, f)
    print(f'tfidf_matrix_{i}.pkl')

z = []
for i in range(12):
    try:
        # Load the TF-IDF vectorized matrix
        with open(f'tfidf_matrix_{i}.pkl', 'rb') as f:
            z.append(pickle.load(f))
            
        print(f'read tfidf_matrix_{i}.pkl')
    except:
        print(f'read tfidf_matrix_{i}.pkl not found')
X = np.vstack(z)  

tfidf_matrix_0.pkl
tfidf_matrix_1.pkl
tfidf_matrix_2.pkl
tfidf_matrix_3.pkl
tfidf_matrix_4.pkl
tfidf_matrix_5.pkl
tfidf_matrix_6.pkl
tfidf_matrix_7.pkl
tfidf_matrix_8.pkl
tfidf_matrix_9.pkl
tfidf_matrix_10.pkl
tfidf_matrix_11.pkl
read tfidf_matrix_0.pkl
read tfidf_matrix_1.pkl
read tfidf_matrix_2.pkl
read tfidf_matrix_3.pkl
read tfidf_matrix_4.pkl
read tfidf_matrix_5.pkl
read tfidf_matrix_6.pkl
read tfidf_matrix_7.pkl
read tfidf_matrix_8.pkl
read tfidf_matrix_9.pkl
read tfidf_matrix_10.pkl
read tfidf_matrix_11.pkl


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_videos_using_clustering(keyword, vectorizer, X, labels, df, top_n=5):
    # Preprocess the keyword
    keyword = preprocess_text(keyword)
    # Preprocess the keyword
    keyword_vector = vectorizer.transform([keyword]).toarray()

    # Calculate the centroid of each cluster
    cluster_centroids = []
    for cluster_id in np.unique(labels):
        cluster_vectors = X[labels == cluster_id]
        cluster_centroid = cluster_vectors.mean(axis=0)
        cluster_centroids.append(cluster_centroid)

    # Find the most similar cluster to the keyword
    cluster_similarities = cosine_similarity(keyword_vector, cluster_centroids)
    most_similar_cluster_id = np.argmax(cluster_similarities)

    # Get all videos in the most similar cluster
    cluster_indices = np.where(labels == most_similar_cluster_id + 1)[0]  # +1 because labels are 1-indexed
    cluster_vectors = X[cluster_indices]

    # Calculate similarity of the keyword to all videos in the cluster
    video_similarities = cosine_similarity(keyword_vector, cluster_vectors)[0]

    # Get the indices of the top N matches within the cluster
    top_indices = np.argsort(video_similarities)[::-1][:top_n]

    # Retrieve the corresponding titles and video URLs
    recommendations = df.iloc[cluster_indices[top_indices]][['title_list', 'video_url_list']]

    return recommendations

# Example usage
keyword = "motorola"
recommendations = recommend_videos_using_clustering(keyword, vectorizer, X, labels, df)
result_title_list = list(recommendations['title_list'])
result_video_url_list = list(recommendations['video_url_list'])
for i, j in zip(result_title_list, result_video_url_list):
  print(i)
  print(j)

Would a Walkman work in space?
https://www.youtube.com/watch?v=TfYeJZeCCjk
RETRO TECH: CAMCORDER
https://www.youtube.com/watch?v=kMtN9KJHn5Y
Analog vs. Digital Cameras
https://www.youtube.com/watch?v=BsAmXFcQMQ4
A look inside the original Macintosh
https://www.youtube.com/watch?v=jdlGvS1y_Ls
Console Wars!
https://www.youtube.com/watch?v=2SkaD7Q53mA
